# EcoWasteClassifier — Тестирование модели локально

Готовая модель для определения типа мусора по фото  
6 классов: КАРТОН • СТЕКЛО • МЕТАЛЛ • БУМАГА • ПЛАСТИК • ОБЫЧНЫЙ МУСОР

In [ ]:
import tensorflow as tf
import os

model_path = "EcoWasteClassifier.h5"

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Модель не найдена! Убедитесь, что файл '{model_path}' лежит рядом с этим ноутбуком.")

model = tf.keras.models.load_model(model_path)
print("Модель EcoWasteClassifier.h5 успешно загружена!")
print(f"Архитектура: {model.name if hasattr(model, 'name') else 'MobileNetV2-based'}")

2025-12-12 15:07:16.412914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765541236.434201  310075 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765541236.440660  310075 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765541236.457290  310075 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765541236.457303  310075 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765541236.457306  310075 computation_placer.cc:177] computation placer alr

Модель EcoWasteClassifier.h5 успешно загружена!
Архитектура: functional


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
try:
    from google.colab import files
    COLAB = True
except:
    COLAB = False

classes_ru = ['КАРТОН', 'СТЕКЛО', 'МЕТАЛЛ', 'БУМАГА', 'ПЛАСТИК', 'ОБЫЧНЫЙ МУСОР']

def predict_waste(image_path=None):
    """
    Предсказывает класс мусора по фото.
    Если image_path=None — предложит загрузить загрузить файл (в Colab) или выбрать вручную.
    """
    if image_path is None:
        if COLAB:
            print("Нажми кнопку и загрузи фото мусора:")
            uploaded = files.upload()
            if not uploaded:
                print("Фото не загружено")
                return
            image_path = list(uploaded.keys())[0]
        else:
            image_path = input("Введи путь к изображению (например, examples/plastic.jpg): ").strip('"\'')
            if not os.path.exists(image_path):
                print("Файл не найден!")
                return

    # Загрузка и подготовка изображения
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Предсказание
    prediction = model.predict(img_array, verbose=0)[0]
    confidence = np.max(prediction) * 100
    idx = np.argmax(prediction)
    label = classes_ru[idx]

    # Цвет заголовка в зависимости от уверенности
    color = 'darkgreen' if confidence > 85 else 'orange' if confidence > 60 else 'red'

    # Вывод результата
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.title(f"{label}\nУверенность: {confidence:.1f}%", 
              fontsize=28, color=color, pad=30, weight='bold')
    plt.axis('off')
    plt.show()

    print(f"Предсказанный тип: {label}")
    print(f"Уверенность модели: {confidence:.1f}%\n")